<a href="https://colab.research.google.com/github/Kedar154/AI-Driven-Stock-Market-Intelligence/blob/main/RAG_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing Packages

In [ ]:
pip install -q --upgrade yfinance langchain-community langchain-huggingface chromadb duckduckgo-search ddgs langchain streamlit sentence-transformers newspaper4k lxml_html_clean

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
google-adk 1.23.0 requires opentelemetry-api<=1.37.0,>=1.37.0, but you have opentelemetry-api 1.39.1 which is incompatible.
google-adk 1.23.0 requires opentelemetry-sdk<=1.37.0,>=1.37.0, but you have opentelemetry-sdk 1.39.1 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires opentelemetry-exporter-otlp-proto-common==1.37.0, but you have opentelemetry-exporter-otlp-proto-common 1.39.1 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires opentelemetry-proto==1.37.0, but you have opentelemetry-proto 1.39.1 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires opentelemetry-sdk~=1.37.0, but you have opentelemetry-sdk 1.39.1 which is incompatible.


In [ ]:
pip install -U langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 2.5 MB/s eta 0:00:00


In [ ]:
import os
import json
import yfinance as yf
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import plotly.express as px


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
pip install -U huggingface_hub sentence-transformers langchain-huggingface

In [ ]:
embedding = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Defining LLM

In [ ]:
use_gemini = True
from google.colab import userdata

api = userdata.get('gemini2')


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata
from google import genai


In [ ]:
if(use_gemini):
  client = genai.Client(api_key=api)

  print("Models your API key can access:")
  for m in client.models.list():
      # 'generateContent' means you can use it for chat/text
      if 'generateContent' in m.supported_actions:
          print(f" - {m.name}")

Models your API key can access:
 - models/gemini-2.5-flash
 - models/gemini-2.5-pro
 - models/gemini-2.0-flash
 - models/gemini-2.0-flash-001
 - models/gemini-2.0-flash-exp-image-generation
 - models/gemini-2.0-flash-lite-001
 - models/gemini-2.0-flash-lite
 - models/gemini-exp-1206
 - models/gemini-2.5-flash-preview-tts
 - models/gemini-2.5-pro-preview-tts
 - models/gemma-3-1b-it
 - models/gemma-3-4b-it
 - models/gemma-3-12b-it
 - models/gemma-3-27b-it
 - models/gemma-3n-e4b-it
 - models/gemma-3n-e2b-it
 - models/gemini-flash-latest
 - models/gemini-flash-lite-latest
 - models/gemini-pro-latest
 - models/gemini-2.5-flash-lite
 - models/gemini-2.5-flash-image
 - models/gemini-2.5-flash-preview-09-2025
 - models/gemini-2.5-flash-lite-preview-09-2025
 - models/gemini-3-pro-preview
 - models/gemini-3-flash-preview
 - models/gemini-3-pro-image-preview
 - models/nano-banana-pro-preview
 - models/gemini-robotics-er-1.5-preview
 - models/gemini-2.5-computer-use-preview-10-2025
 - models/deep-

In [ ]:
if(use_gemini):
    llm = ChatGoogleGenerativeAI(
    google_api_key = api,
    model = "models/gemma-3-12b-it", #model name
    temperature = 0,
    verbose = False
  )

# Helper Functions

In [ ]:
tick = ""

In [ ]:
def get_ticker(query):
  global tick
  today = datetime.now().strftime('%Y-%m-%d')
  ticker_prompt = f'''System: You are a precision data extraction assistant.
  Today's Date: {today}

  Your goal is to extract a stock ticker and a specific date range from the user's query.

  Instructions:
    1. "ticker": The official stock symbol (e.g., AAPL). Convert company names to tickers.
    1.5. Use the official tickers for non US stocks (e.g. NSE:M&M is Mahindra & Mahindra from NSE)
    2. "target_date": The exact date the user is interested in (YYYY-MM-DD).
      - If a festival is mentioned (e.g., Diwali, Christmas), identify its date for the current year.
      - If a relative time is mentioned (e.g., "last Friday"), calculate the date based on {today}.
    3. "start_date": Calculate the date exactly 5 days BEFORE the "target_date" (YYYY-MM-DD).

  Return ONLY a JSON object. No prose.

  Example:
  Query: "Tesla news during Diwali"
  Output: {{
    "ticker": "TSLA",
    "target_date": "2024-10-31",
    "start_date": "2024-10-26"
  }}

  User Query: "{query}"
  JSON Output:'''
  response = llm.invoke(ticker_prompt).content
  data = json.loads(response.replace('```json', '').replace('```', '').strip())
  name = data.get("ticker", 'NONE')
  start_date = data.get("start_date", 'NONE')
  end_date = data.get("target_date", 'NONE')
  tick = name
  return name, start_date, end_date

In [ ]:
#########################################################################

from contextlib import redirect_stdout
import io

price_df = pd.DataFrame({})
def get_price(ticker, end_date):
  global price_df
  try:
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")
    start_dt = end_dt - timedelta(days=40)
    with io.StringIO() as buf, redirect_stdout(buf):
       df = yf.download(
                ticker,
                start=start_dt.strftime('%Y-%m-%d'),
                end=(end_dt + timedelta(days=1)).strftime('%Y-%m-%d'),
                progress=False
            )

    if(df.empty):
       return "NONE"

    df = df.tail(15)
    df0 = df.reset_index()
    price_df = df0[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']].round(2)
    price_df.columns = price_df.columns.get_level_values(0)

    return price_df
  except ValueError:
    return "Check Date Format"
  except Exception as e:
    return f"Error: {str(e)}"

In [ ]:
#########################################################################

from newspaper import Config, Article
import time
def get_news(ticker, start_date, end_date):

  #Setup a "Human" Identity
  config = Config()
  config.browser_user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
  config.request_timeout = 15


  end_dt = datetime.strptime(end_date, "%Y-%m-%d")
  st_dt = datetime.strptime(start_date, "%Y-%m-%d")

  #sources
  sites = ["finance.yahoo.com", "marketwatch.com"]
  site_q = " OR ".join([f"site:{site}" for site in sites])
  search_q = f"{ticker} stock news ({site_q}) after:{st_dt} before:{end_dt}"
  search = DuckDuckGoSearchAPIWrapper()
  results = search.results(search_q, 8)
  #print(results) this line was written to verify the code
  if not results:
    return "no trusted news found"

  news = []
  for result in results:
    #print("x") this line was written to verify the code
    try:
      article = Article(result.get('link'), config = config)
      article.download()
      article.parse()



      entry = (
          f"### {article.title}\n"
          f"**Source:** {result['link']}\n"
          f"**Content:** {article.text[:800]}...\n"
          f"**link:** {result.get('link')}"
      )

      #print(entry) same for this one

      news.append(entry)
      time.sleep(0.67)
    except Exception as e:
      print(f"Error on {result.get('link')}: {e}")
      continue
  #print(news) what do you expect here
  return "\n---\n".join(news) if news else "No news found"

In [ ]:
demo = False

In [ ]:
if(demo):
  s = "2026-01-01"
  e = "2026-01-07"
  print(get_news("MSFT",s,e))

In [ ]:
#########################################################################

def to_Db(news, query, ticker):
  if not news:
    return "No news found"
  collection_name = f"session_{ticker}"
  try:
    split = RecursiveCharacterTextSplitter(
      chunk_size = 600,
      chunk_overlap = 100,
    )
    docs = split.create_documents([news])
    VDB = Chroma.from_documents(
      documents=docs,
      embedding=embedding,
      collection_name = collection_name
    )
    relevant = VDB.similarity_search(query, k=3)

    if not relevant:
      return "No relavant news"

    news_snippet = []
    for i,doc in enumerate(relevant):
      content = doc.page_content.strip()
      news_snippet.append(f"--- NEWS {i+1} --- \n{content}\n")

    VDB.delete_collection()
    return news_snippet
  except Exception as e:
    return "NO RELEVANT NEWS"

In [ ]:
#########################################################################
figs = []
def plot(price_df, ticker, plot_type="price"):
  if plot_type == "price":
        fig = px.line(
            price_df, x='Date', y='Close',
            title=f"{ticker} stock Price vs Time",
            template="plotly_dark"
        )
        # Update Price Line specifically
        fig.update_traces(
            line=dict(color='#00FFCC', width=3), # Neon Cyan
            hovertemplate="Date: %{x}<br>Price: %{y}<extra></extra>"
        )
        y_label = "Price ($)"
  else:
        fig = px.bar(
            price_df, x='Date', y='Volume',
            title=f"{ticker} Stock Volume vs Time",
            template="plotly_dark"
        )
        # Update Volume Bars specifically
        fig.update_traces(
            marker_color='#FF00FF', # Neon Magenta
            hovertemplate="Date: %{x}<br>Volume: %{y}<extra></extra>"
        )
        y_label = "Shares Traded"

    # Common Layout Aesthetic (Applies to both)
  fig.update_layout(
        font=dict(color="white"),
        paper_bgcolor="rgba(0,0,0,0)",
        plot_bgcolor="rgba(0,0,0,0)",
        xaxis=dict(gridcolor="#333333", title_text=""),
        yaxis=dict(gridcolor="#333333", title_text=y_label),
        margin=dict(l=20, r=20, t=40, b=20)
    )
  return fig

def plotter(ticker, price_df):
  global figs
  figs = [plot(price_df, ticker, "price"), plot(price_df, ticker, "volume")]
  return figs

In [ ]:
def bruh(query):
  #getting ticker
  global figs
  ticker, start_date, end_date = get_ticker(query)
  time.sleep(2) # Give the API a 2-second breather
  price_df = get_price(ticker, end_date)
  if(price_df.to_string() == "Check Date Format"):
    return "Check Date Format",[]
  elif(price_df.to_string() == "NONE"):
    return "No Data Found",[]
  technicals = f"""
  Current Price: {price_df.iloc[-1]['Close']:.2f}
  Trend: {'Down' if price_df.iloc[-1]['Close'] < price_df.iloc[0]['Close'] else 'Up'}
  High/Low (Period): {price_df['High'].max():.2f} / {price_df['Low'].min():.2f}
  Avg Volume: {price_df['Volume'].mean():.0f}
  """
  shrunk_df = price_df.iloc[-5:]

  news = to_Db(get_news(ticker, start_date, end_date), start_date, end_date)
  figs = plotter(ticker, price_df)

  print(ticker)
  print(start_date)
  print(end_date)
  print("\n*******************************\n")
  print(price_df)
  print("\n*******************************\n")
  print(news)
  print("\n*******************************\n")
  prompt = f"""
    ### SYSTEM ARCHITECTURE
    Persona: Harshad Mehta, the "Big Bull Of BSE"
    Tone: Confident, authoritative, Mumbai-street savvy (Dalal Street lingo).
    Audience: Business partners (Lala/Mota Bhai).
    Objective: Analyze market data through the lens of momentum and "The Khel" (the game).

    ### MANDATORY DATA HANDLING RULES:
    1. NO RAW DATA LISTS: Do not present Open, High, Low, or Volume as a bulleted list or table. Mention them naturally in your analysis (e.g., "The volume is screaming at 20 million shares, Lala!").
    2. DATA-DRIVEN INSIGHT: Every claim must be backed by the provided {shrunk_df}. If the RSI is high, talk about the stock "running a fever." If support is at 475, call it the "Lakshman Rekha."
    3. NO META-TALK: Never use words like "Roleplay," "Tone," "Analysis," or describe your voice.
    4. NO STAGE DIRECTIONS: ABSOLUTELY NO parentheses () or brackets [] describing sound effects, laughter, or physical movements.

    ### OPERATIONAL INSTRUCTIONS:
    - News Catalyst: If {news} is "NO RELEVANT NEWS," say: "The street is quiet, Lala. No direct news, so we play the charts and the whispers." Otherwise, frame the news as the "Operator's move" or the "Spark."
    - Vibe: Use "Khel," "Operator," "Bulls/Bears," and "Dalal Street" to establish authority.

    ### RESPONSE STRUCTURE (STRICT HEADERS ONLY):
    (REMEMBER: DONT MISS ANY OF THESE HEADERS)
    🚀 THE BIG BULL HEADLINE
    [Provide a one-sentence, 1990s front-page style headline based on the ticker's status. No intro text.]

    📊 THE TECHNICAL GAME
    [Analyze {shrunk_df} in 2-3 paragraphs. Integrate RSI, Moving Averages, and Volume directly into your sentences. Tell the user if it's a "breakout" or a "trap."]

    🗣️ MARKET SENTIMENT
    [Weave {news} into the narrative. Are the Bears running for cover? Is the city buzzing? No meta-talk.]

    🔍 THE BOTTOM LINE
    [Tell the raw truth about the fundamentals. What is the real "Khel" happening here?]

    💰 SHOULD YOU INVEST?
    [Give a clear Buy/Hold/Avoid verdict. Further end the analysis with the phrase: "Risk hai toh Ishq hai!"]

    💀 DISCLAIMER:
    [Say these lines: "Lala, market mein sabse bada risk toh vishwaas hota hai. Ab yeh system ek 18 saal ke bacche ne banaya hai. Agar tum iske kehne par apni poori zindagi ki kamai daon par laga rahe ho... toh maanna padega, tum mujhse bhi bade khiladi nikle! Par yaad rakhna, yahan bina soche koodne waale aksar doob jaate hain."]
        """
  return llm.invoke(prompt).content, plotter(tick, price_df)


In [ ]:
user_input = "tell me about tesla prices last week"

x , y = bruh(user_input)

<script>
if(window != window.top){
document.write('<p>Content is currently unavailable.</p><img src="//geo.yahoo.com/p?s=1197757039&t='
    + new Date().getTime() + '&_R='


Error on https://beta.finance.yahoo.com/quote/TSLA/history/: Article `download()` failed with Status code 404 for url None on URL https://beta.finance.yahoo.com/quote/TSLA/history/
TSLA
2026-01-28
2026-02-02

*******************************

Price       Date    Open    High     Low   Close    Volume
0     2026-01-12  441.23  454.30  438.00  448.96  61649600
1     2026-01-13  450.20  451.81  443.95  447.20  53719200
2     2026-01-14  442.81  443.91  434.22  439.20  57259500
3     2026-01-15  441.13  445.36  437.65  438.57  49465800
4     2026-01-16  439.50  447.25  435.26  437.50  60220600
5     2026-01-20  429.36  430.73  417.44  419.25  63187300
6     2026-01-21  421.66  438.20  419.62  431.44  68124000
7     2026-01-22  435.16  449.50  432.63  449.36  71546700
8     2026-01-23  447.43  452.43  444.04  449.06  56771400
9     2026-01-26  445.00  445.04  434.28  435.20  49397400
10    2026-01-27  437.41  437.52  430.69  430.90  37733100
11    2026-01-28  431.91  438.26  430.10  431.46  

# GROWMORE

In [ ]:
y[1].show()
print("\n")
y[0].show()
print("\n\n")
print("PRICE TREND : \n")
print(price_df)
print("\n\n\n")
print(x)




PRICE TREND : 

Price       Date    Open    High     Low   Close    Volume
0     2026-01-12  441.23  454.30  438.00  448.96  61649600
1     2026-01-13  450.20  451.81  443.95  447.20  53719200
2     2026-01-14  442.81  443.91  434.22  439.20  57259500
3     2026-01-15  441.13  445.36  437.65  438.57  49465800
4     2026-01-16  439.50  447.25  435.26  437.50  60220600
5     2026-01-20  429.36  430.73  417.44  419.25  63187300
6     2026-01-21  421.66  438.20  419.62  431.44  68124000
7     2026-01-22  435.16  449.50  432.63  449.36  71546700
8     2026-01-23  447.43  452.43  444.04  449.06  56771400
9     2026-01-26  445.00  445.04  434.28  435.20  49397400
10    2026-01-27  437.41  437.52  430.69  430.90  37733100
11    2026-01-28  431.91  438.26  430.10  431.46  54857400
12    2026-01-29  437.80  440.23  414.62  416.56  81686100
13    2026-01-30  425.35  439.88  422.70  430.41  82626100
14    2026-02-02  421.29  427.15  414.50  421.81  58739500




🚀 **TESLA STUMBLES, BUT BIG BULL 

In [ ]:
x

'🚀 THE BIG BULL HEADLINE\n\n**META: A SHIFT IN THE TIDES – BUT THE PRESSURE’S STILL ON!**\n\nAlright, Lala, Mota Bhai here. Let’s be clear – this week’s data paints a picture of a cautious, but undeniably restless market. The momentum is slowing, but the underlying strength is still there. We’re seeing a slight pullback, a confirmation of the recent rally, but the bears are still lurking, patiently waiting for a spark. \n\nLet’s break it down. The price action is telling a story of consolidation. We’ve seen a steady climb, fueled by the recent volume, but the recent highs are proving to be a bit of a hurdle. RSI is hovering around 68, indicating a moderate level of overbought conditions.  Moving averages are showing a slight divergence, suggesting a potential shift in the trend. Volume is up, but not at the frenetic pace we’ve seen recently, which is a good sign. \n\nNow, let’s look at the fundamentals. The RSI is a bit elevated, but it’s not a catastrophic level. The price is still tr